# Multi-party Record Linkage with Blocking

In this tutorial, we will demonstrate the CLI tools for multiparty record linkage with blocking techniques. 


In [1]:
import io
import os
import math
from bitarray import bitarray
import base64
import time
from IPython import display

import anonlink
import pandas as pd
import clkhash
from clkhash import clk

from client.utils import combine_clks_blocks

In [2]:
import json
import anonlink
from collections import defaultdict


def deserialize_bitarray(bytes_data):
    ba = bitarray(endian='big')
    data_as_bytes = base64.decodebytes(bytes_data.encode())
    ba.frombytes(data_as_bytes)
    return ba


def deserialize_filters(filters):
    res = []
    for i, f in enumerate(filters):
        ba = deserialize_bitarray(f)
        res.append(ba)
    return res


def solve(encodings, rec_to_blocks, threshold: float = 0.8):
    """ entity resolution, baby

    calls anonlink to do the heavy lifting.

    :param encodings: a sequence of lists of Bloom filters (bitarray). One for each data provider
    :param rec_to_blocks: a sequence of dictionaries, mapping a record id to the list of blocks it is part of. Again,
                          one per data provider, same order as encodings.
    :param threshold: similarity threshold for solving
    :return: same as the anonlink solver.
             An sequence of groups. Each group is an sequence of
             records. Two records are in the same group iff they represent
             the same entity. Here, a record is a two-tuple of dataset index
             and record index.
    """
    def my_blocking_f(ds_idx, rec_idx, _):
        return rec_to_blocks[ds_idx][rec_idx]

    candidate_pairs = anonlink.candidate_generation.find_candidate_pairs(
        encodings,
        anonlink.similarities.dice_coefficient,
        threshold=threshold,
        blocking_f=my_blocking_f)
    # Need to use the probabilistic greedy solver to be able to remove the duplicate. It is not configurable
    # with the native greedy solver.
    return anonlink.solving.probabilistic_greedy_solve(candidate_pairs, merge_threshold=1.0)


def naive_solve(encodings, threshold: float = 0.8):
    """ entity resolution, baby

    calls anonlink to do the heavy lifting.

    :param encodings: a sequence of lists of Bloom filters (bitarray). One for each data provider
    :param threshold: similarity threshold for solving
    :return: same as the anonlink solver.
             An sequence of groups. Each group is an sequence of
             records. Two records are in the same group iff they represent
             the same entity. Here, a record is a two-tuple of dataset index
             and record index.
    """
    candidate_pairs = anonlink.candidate_generation.find_candidate_pairs(
        encodings,
        anonlink.similarities.dice_coefficient,
        threshold=threshold)
    # Need to use the probabilistic greedy solver to be able to remove the duplicate. It is not configurable
    # with the native greedy solver.
    return anonlink.solving.probabilistic_greedy_solve(candidate_pairs, merge_threshold=1.0)


def evaluate(found_groups, true_matches):
    tp = len([x for x in found_groups if x in true_matches])
    fp = len([x for x in found_groups if x not in true_matches])
    fn = len([x for x in true_matches if x not in found_groups])

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return precision, recall


def reduction_ratio(filtered_reverse_indices, data, K):
    """Assess reduction ratio for multiple parties."""
    naive_num_comparison = 1
    for d in data:
        naive_num_comparison *= len(d)

    block_keys = defaultdict(int)  # type: Dict[Any, int]
    for reversed_index in filtered_reverse_indices:
        for key in reversed_index:
            block_keys[key] += 1
    final_block_keys = [key for key, count in block_keys.items() if count >= K]

    reduced_num_comparison = 0
    for key in final_block_keys:
        num_comparison = 1
        for reversed_index in filtered_reverse_indices:
            index = reversed_index.get(key, [0])
            num_comparison *= len(index)
        reduced_num_comparison += num_comparison
    rr = 1 - reduced_num_comparison / naive_num_comparison
    return rr, reduced_num_comparison, naive_num_comparison


def set_completeness(filtered_reverse_indices, truth, K):
    """Assess reduction ratio for multiple parties."""
    block_keys = defaultdict(int)  # type: Dict[Any, int]
    for reversed_index in filtered_reverse_indices:
        for key in reversed_index:
            block_keys[key] += 1
    final_block_keys = [key for key, count in block_keys.items() if count >= K]

    sets = defaultdict(set)
    for i, reversed_index in enumerate(filtered_reverse_indices):
        for key in final_block_keys:
            index = reversed_index.get(key, None)
            if index is not None:
                for ind in index:
                    sets[key].add((i, ind))

    num_true_matches = 0
    for true_set in truth:
        check = False
        true_set = set(true_set)
        for s in sets.values():
            if true_set.intersection(s) == true_set:
                check = True
        if check:
            num_true_matches += 1

    sc = num_true_matches / len(truth)
    return sc

Suppose we are interested to find records that appear at least twice in 3 parties

## Generate CLKs and Candidate Blocks

First we have a look at dataset

In [3]:
corruption_rate = 20
file_template = 'data/ncvr_numrec_5000_modrec_2_ocp_' + str(corruption_rate) + '_myp_{}_nump_10.csv'
df1 = pd.read_csv(file_template.format(0))
df1.head()

,recid,givenname,surname,suburb,postcode
0,1503359,pauline,camkbell,lilescille,28091
1,1972058,deborah,galyen,ennike,286z3
2,889525,charle5,mitrhell,roaring river,28669
3,4371845,petehr,werts,swannanoa,28478
4,1187991,katpy,silbiger,duyham,27705


A hashing schema instructs clkhash how to treat each column for generating CLKs. A detailed description of the hashing schema can be found in the api docs. We will ignore the columns ‘ENTID’ for CLK generation.

In [4]:
with open("novt_schema.json") as f:
    print(f.read())


{
  "version": 3,
  "clkConfig": {
    "l": 1024,
    "kdf": {
      "type": "HKDF",
      "hash": "SHA256",
      "salt": "SCbL2zHNnmsckfzchsNkZY9XoHk96P/G5nUBrM7ybymlEFsMV6PAeDZCNp3rfNUPCtLDMOGQHG4pCQpfhiHCyA==",
      "info": "c2NoZW1hX2V4YW1wbGU=",
      "keySize": 64
    }
  },
  "features": [
        {
      "identifier": "recid",
      "ignored": true
    },
    {
      "identifier": "givenname",
      "format": {
        "type": "string",
        "encoding": "utf-8",
        "maxLength": 30,
        "case": "lower"
      },
      "hashing": {
        "comparison": {"type":  "ngram", "n":  2},
        "strategy": {"bitsPerFeature":  100},
        "hash": {"type": "blakeHash"},
        "missingValue": {
          "sentinel": ".",
          "replaceWith": ""
        }
      }
    },
    {
      "identifier": "surname",
      "format": {
        "type": "string",
        "encoding": "utf-8",
        "maxLength": 30,
        "case": "lower"
      },
      "hashing": {
        "comp

### Validate the schema
The command line tool can check that the linkage schema is valid:

In [5]:
!anonlink validate-schema "novt_schema.json"

schema is valid


### Hash data
We can now hash our Personally Identifiable Information (PII) data from the CSV file using our defined linkage schema. We must provide two secret keys to this command - these keys have to be used by both parties hashing data. For this toy example we will use the secret ‘secret’, for real data, make sure that the secret contains enough entropy, as knowledge of this secret is sufficient to reconstruct the PII information from a CLK!

In [6]:
!anonlink hash 'data/ncvr_numrec_5000_modrec_2_ocp_20_myp_0_nump_10.csv' secret 'novt_schema.json' 'novt_clk_0.json'

CLK data written to novt_clk_0.json


Let's hash data for party B and C:

In [7]:
!anonlink hash 'data/ncvr_numrec_5000_modrec_2_ocp_20_myp_1_nump_10.csv' secret 'novt_schema.json' 'novt_clk_1.json'

CLK data written to novt_clk_1.json


In [8]:
!anonlink hash 'data/ncvr_numrec_5000_modrec_2_ocp_20_myp_2_nump_10.csv' secret 'novt_schema.json' 'novt_clk_2.json'

CLK data written to novt_clk_2.json


In [9]:
!anonlink describe 'novt_clk_0.json'

    ----------------------------------------------------------------------------------------------------------------------------
    |                                                        popcounts                                                         |
    ----------------------------------------------------------------------------------------------------------------------------

 298|                                     o                        
 282|                                   o ooooo                    
 267|                                   o ooooo                    
 251|                                   ooooooo                    
 235|                                 ooooooooo o                  
 220|                                 ooooooooo o                  
 204|                                 ooooooooo o                  
 188|                               o ooooooooo oo                 
 173|                               ooooooooooo oo o               


### Block dataset


Blocking is a technique that makes record linkage scalable. It is achieved by partitioning datasets into groups, called blocks and only comparing records in corresponding blocks. This can reduce the number of comparisons that need to be conducted to find which pairs of records should be linked.

In [10]:
with open("blocking_schema.json") as f:
    print(f.read())

{
    "type": "lambda-fold",
    "version": 1,
    "config": {
        "blocking-features": [1, 2],
        "Lambda": 25,
        "bf-len": 2048,
        "num-hash-funcs": 10,
        "K": 30,
        "input-clks": true,
        "random_state": 0
    }
}


**Party A Hash its Data**

In [11]:
!anonlink block 'novt_clk_0.json' 'blocking_schema.json' 'novt_blocks_0.json'

Number of Blocks:   25
Minimum Block Size: 5000
Maximum Block Size: 5000
Average Block Size: 5000
Median Block Size:  5000
Standard Deviation of Block Size:  0.0


**Party B Hash its Data**

In [12]:
!anonlink block 'novt_clk_1.json' 'blocking_schema.json' 'novt_blocks_1.json'

Number of Blocks:   25
Minimum Block Size: 5000
Maximum Block Size: 5000
Average Block Size: 5000
Median Block Size:  5000
Standard Deviation of Block Size:  0.0


**Party C Hash its Data**

In [13]:
!anonlink block 'novt_clk_2.json' 'blocking_schema.json' 'novt_blocks_2.json'

Number of Blocks:   25
Minimum Block Size: 5000
Maximum Block Size: 5000
Average Block Size: 5000
Median Block Size:  5000
Standard Deviation of Block Size:  0.0


## Get Ground Truth

In [14]:
truth = []

for party in [0, 1, 2]:
    df = pd.read_csv('data/ncvr_numrec_5000_modrec_2_ocp_20_myp_{}_nump_10.csv'.format(party))
    truth.append(pd.DataFrame({'id{}'.format(party): df.index, 'recid': df['recid']}))
    
dfj = truth[0].merge(truth[1], on='recid', how='outer')
for df in truth[2:]:
    dfj = dfj.merge(df, on='recid', how='outer')

dfj = dfj.drop(columns=['recid'])
true_matches = set()
for row in dfj.itertuples(index=False):
    cand = [(i, int(x)) for i, x in enumerate(row) if not math.isnan(x)]
    if len(cand) > 1:
        true_matches.add(tuple(cand))

print(f'we have {len(true_matches)} true matches')
e = iter(true_matches)
for i in range(10):
    print(next(e))

we have 1649 true matches
((0, 159), (1, 169), (2, 169))
((1, 2309), (2, 2137))
((0, 366), (1, 589), (2, 362))
((0, 3719), (1, 3701), (2, 1560))
((0, 182), (1, 758), (2, 760))
((0, 3886), (1, 3865), (2, 311))
((0, 2878), (2, 280))
((0, 498), (1, 2269), (2, 492))
((0, 3630), (2, 282))
((1, 692), (2, 374))


## Solve with Anonlink

In [15]:
clk_files = ['novt_clk_{}.json'.format(x) for x in range(3)]
block_files = ['novt_blocks_{}.json'.format(x) for x in range(3)]

clk_blocks = []

for i, (clk_f, block_f) in enumerate(zip(clk_files, block_files)):
    print('Combining CLKs and Blocks for Party {}'.format(i))
    clk_blocks.append(combine_clks_blocks(open(clk_f, 'rb'), open(block_f, 'rb')))
    
    
clk_groups = []
rec_to_blocks = {}

for i, clk_blk in enumerate(clk_blocks):
    clk_groups.append(deserialize_filters([r[0] for r in clk_blk]))
    rec_to_blocks[i] = {rind: clk_blk[rind][1:] for rind in range(len(clk_blk))}


Combining CLKs and Blocks for Party 0
Combining CLKs and Blocks for Party 1
Combining CLKs and Blocks for Party 2


## Assess Precision and Recall

In [16]:
threshold = 0.87

# matching with blocking
found_groups = solve(clk_groups, rec_to_blocks, threshold)
print("Example found groups: ")
for i in range(10):
    print(found_groups[i])
precision, recall = evaluate(found_groups, true_matches)
print('\n\nWith blocking: ')
print(f'precision: {precision}, recall: {recall}')

# matching without blocking
found_groups = naive_solve(clk_groups, threshold)
precision, recall = evaluate(found_groups, true_matches)
print('Without blocking: ')
print(f'precision: {precision}, recall: {recall}')

Example found groups: 
((1, 273), (2, 3985))
((1, 2572), (2, 4597))
((1, 2772), (2, 3127), (0, 153))
((1, 3610), (2, 1652), (0, 231))
((0, 2973), (1, 4718), (2, 2278))
((0, 2878), (2, 280))
((0, 4995), (1, 2634), (2, 303))
((0, 3907), (1, 829))
((0, 1993), (1, 2924), (2, 2622))
((1, 6), (2, 8))


With blocking: 
precision: 0.7808661926308985, recall: 0.7325651910248635
Without blocking: 
precision: 0.7808661926308985, recall: 0.7325651910248635


## Assess Reduction Ratio

**Reduction Ratio**

Reduction ratio measures the proportion of number of comparisons reduced by using blocking technique. If we have two data providers each has $N$ number of records, then 

$$\text{reduction ratio}= 1 - \frac{\text{number of comparisons after blocking}}{N^3}$$


In [17]:
import json
import pandas as pd
from blocklib import assess_blocks_2party

block_a = json.load(open('novt_blocks_0.json'))['blocks']
block_b = json.load(open('novt_blocks_1.json'))['blocks']
block_c = json.load(open('novt_blocks_2.json'))['blocks']

filtered_reverse_indices = [block_a, block_b, block_c]

data = []
for party in [0, 1, 2]:
    dfa = pd.read_csv('data/ncvr_numrec_5000_modrec_2_ocp_0_myp_{}_nump_10.csv'.format(party))
    recid = dfa['recid'].values
    data.append(recid)
    

rr, reduced_num_comparison, naive_num_comparison = reduction_ratio(filtered_reverse_indices, data, K=2)
print('\nWith blocking, we reduced {:,} comparisons to {:,} comparisons i.e. the reduction ratio={}'
      .format(naive_num_comparison, reduced_num_comparison, rr))

TypeError: unhashable type: 'dict'

## Assess Set Completeness

**Set Completeness**

Set completeness (aka pair completeness in two-party senario) measure how many true matches are maintained after blocking. It is evalauted as

$$\text{set completeness}= \frac{\text{number of true matches after blocking}}{\text{number of all true matches}}$$


In [ ]:
sc = set_completeness(filtered_reverse_indices, true_matches, K=2)
print('Set completeness = {}'.format(sc))